In [2]:
import cv2
import numpy as np

In [3]:
# Указываем путь до видео, конфига и файла с весами
video_name = "G:/Programming/JML_contest/jml_sources/rat.mp4"
cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg"
weights_file_name = "yolov4-tiny-custom_last.weights"

In [ ]:
# Загрузка YOLO
net = cv2.dnn.readNet(weights_file_name, cfg_file_name)
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

In [5]:
# Загрузка имен классов
with open('yolov4-tiny/obj.names', "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [32]:
# Установка захвата видео
cap = cv2.VideoCapture(video_name)
# video = cv2.VideoWriter("G:/Programming/JML_contest/jml_sources/rat_detected.mp4", -1, 1, (800, 600))

# Массив координат обнаруженного объекта
coords = []

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (800, 600))
    height, width, channels = frame.shape



    # Обработка изображения YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Показываем информацию на экране
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Объект обнаружен
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Координаты прямоугольника
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (0,255,0)  # Зеленый цвет
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), font, 1, color, 2)

    # Добавляем координаты объекта в массив
    coords.append((center_x, center_y))
    if len(coords) < 1:
        pass
    else:
        for i in range(len(coords)-1):
            cv2.line(frame, coords[i], coords[i+1], (0, 255, 0), thickness=2)
    
    
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:  # Нажмите ESC для выхода
        break

# video.release()
cap.release()
cv2.destroyAllWindows()